<a href="https://colab.research.google.com/github/sujalgawas/brain_tumour/blob/main/rick_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --quiet --upgrade

In [ ]:
import opendatasets as od

data = "https://www.kaggle.com/datasets/andradaolteanu/rickmorty-scripts/data?select=RickAndMortyScripts.csv"

od.download(data)

Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/rickmorty-scripts


100%|██████████| 66.2k/66.2k [00:00<00:00, 10.8MB/s]

In [ ]:
import pandas as pd
import torch
from torch import nn
from typing import Dict,List,Tuple
from torch.utils.data import DataLoader,Dataset,RandomSampler,SequentialSampler
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from transformers import(
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    PreTrainedTokenizer,
    PreTrainedModel,
    get_linear_schedule_with_warmup,
)
from tqdm.notebook import tqdm,trange
import os
import pickle

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data_dir = "/content/rickmorty-scripts/RickAndMortyScripts.csv"

rick_data = pd.read_csv(data_dir)

In [ ]:
rick_data.head()

,index,season no.,episode no.,episode name,name,line
0,0,1,1,Pilot,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,1,1,1,Pilot,Morty,"What, Rick? What’s going on?"
2,2,1,1,Pilot,Rick,"I got a surprise for you, Morty."
3,3,1,1,Pilot,Morty,It's the middle of the night. What are you tal...
4,4,1,1,Pilot,Rick,"Come on, I got a surprise for you. Come on, h..."


In [ ]:
character = "Rick"
context = []
response = []
indices = []

# Find indices where the character matches
for index, name in enumerate(rick_data["name"]):
    if name == character:
        indices.append(index)

# Separate lines based on indices
for index, line in enumerate(rick_data["line"]):
    if index in indices:
        response.append(line)
    else:
        context.append(line)

In [ ]:
print(response[:5])

["Morty! You gotta come on. Jus'... you gotta come with me.", 'I got a surprise for you, Morty.', 'Come on, I got a surprise for you.  Come on, hurry up.', 'We gotta go, gotta get outta here, come on. Got a surprise for you Morty.', 'What do you think of this... flying vehicle, Morty? I built it outta stuff I found in the garage.']


In [ ]:
print(context[:5])

['What, Rick? What’s going on?', "It's the middle of the night. What are you talking about?", "Ow! Ow! You're tugging me too hard!", "Yeah, Rick... I-it's great. Is this the surprise?", 'What?! A bomb?!']


In [ ]:
import pandas as pd

# Ensure response and context are lists
response = [response] if isinstance(response, str) else response
context = [context] if isinstance(context, str) else context

# Calculate number of rows needed
num_rows = max(len(response), (len(context) + 5) // 6)

# Create data dictionary
data = {
    "response": [],
    "context/0": [], "context/1": [], "context/2": [],
    "context/3": [], "context/4": [], "context/5": []
}

for i in range(num_rows):
    # Append response, repeating values if necessary
    data["response"].append(response[i % len(response)])

    # Append context, repeating values if necessary
    for j in range(6):
        idx = i * 6 + j
        if idx < len(context):
            data[f"context/{j}"].append(context[idx])
        else:
            # If we've run out of context, repeat from the beginning
            data[f"context/{j}"].append(context[idx % len(context)])

# Create DataFrame
df = pd.DataFrame(data)

In [ ]:
df.tail()

,response,context/0,context/1,context/2,context/3,context/4,context/5
415,You don’t have to be a dick.,"Well, Morty, if you know must know, the syring...","Enough. Kids, it's time to go. We don't want t...","W-w-what are you doing there, Beth? What are y...","Well, I mean, you don't want to get pierced by...",Can't argue with that.,Great. We'll see you later.
416,"Well, you can keep wondering that while we go ...","Hey, hey, be careful with that. It's for somet...","Whoa, whoa, whoa, whoa! I know what it looks l...","Whoa! Oh! Whoa, whoa, whoa!","Oh, crap, that sun is bright.","Okay, come on. This can't really be the way I ...",So... hot. This is how I'm gonna die.
417,"Whoa!! Hahaha, yeah! Atlantis, baby!","Oh, God, moisture.","Oh, God, the moisture! Dial it back, God! Dial...",Oh! Oh!,"Oh, God! Perpendicular, perpendicular!","Oh, shiiit! Oh! Oh! Oh!",Come on. That's it.
418,Got some of that mermaid puss!,Come get this delicious brine.,"Aah! Come on! Come on, motherfucker! Come on!",Yes!,How is this even family therapy if Dad's not i...,"Yeah, and what's courageous about eating a hot...","It's nobody's choice to be here, you knobs. Th..."
419,"Pssh! Not at all, Morty. That place will never...","Oh, the Smith family, minus a dad. You're pati...",Temporarily. By order of the school.,Me too. How long have you all been eating poop?,We... have never... eaten poop.,"Uh, me, neither. Say, where did my family get ...","Smith family, I'm Dr. Wong. Come on in."


In [ ]:
trn_df, val_df = train_test_split(df, test_size=0.1)

In [ ]:
from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

#tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
#model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")


tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')
model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def train(train_dataset,model:model,tokenizer:tokenizer):

  batch_size = 2

  def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

  train_sampler = RandomSampler(train_dataset)

  train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler,batch_size=batch_size, collate_fn=collate, drop_last = True
    )

  t_total = len(train_dataloader) // 1 * 40

  model.shared(torch.tensor(len(tokenizer)).to(device))

  no_decay = ["bias", "LayerNorm.weight"]
  optimizer_grouped_parameters = [
      {
          "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
          "weight_decay": 0.0,
      },
      {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
  ]
  optimizer = AdamW(optimizer_grouped_parameters,lr=5e-5,eps=1e-8)
  schedular = get_linear_schedule_with_warmup(
      optimizer,num_warmup_steps=0,num_training_steps=t_total
  )

  global_step = 0
  epochs_trained = 0
  steps_trained_in_current_epoch = 0

  tr_loss, logging_loss = 0.0,0.0
  train_iterator = trange(
      epochs_trained,int(40),desc="Epoch"
  )
  for _ in train_iterator:
    epochs_iterator = tqdm(
        train_dataloader,desc="Iteration"
    )
    for step,batch in enumerate(epochs_iterator):

      inputs,labels = (batch,batch)
      if inputs.shape[1] > 1024:continue
      inputs=inputs.to(device)
      labels = labels.to(device)
      model.train()
      outputs = model(inputs,labels=labels)
      loss = outputs[0]
      loss.backward()
      tr_loss += loss.item()
      schedular.step()  # Update learning rate schedule
      model.zero_grad()
      global_step += 1
  return global_step, tr_loss / global_step


In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer,df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = 'cached'
        cached_features_file = os.path.join(
            directory, 't5-small' + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file):
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


In [ ]:
def load_and_cache_examples(tokenizer,df_trn,val_df,evaluate=False):
  return ConversationDataset(tokenizer,val_df if evaluate else df_trn)

  def set_seed():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)

In [ ]:
def evaluate(model:model,tokenizer:tokenizer,df_trn,df_val):
  eval_dataset = load_and_cache_examples(tokenizer,df_trn,df_val,evaluate=True)
  os.makedirs('./output-large',exist_ok=True)

  def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

  eval_sampler = SequentialSampler(eval_dataset)
  eval_dataloader = DataLoader(
      eval_dataset, sampler=eval_sampler, batch_size=2, collate_fn=collate, drop_last = True
  )

  eval_loss = 0.0
  nb_eval_steps = 0
  model.eval()

  for batch in tqdm(eval_dataloader,desc="Evaluating"):
    inputs,labels = (batch,batch)
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.inference_mode():
      outputs = model(inputs,labels=labels)
      lm_loss = outputs[0]
      eval_loss += lm_loss.mean().item()
    nb_eval_steps += 1

  eval_loss = eval_loss / nb_eval_steps
  perplexity = torch.exp(torch.tensor(eval_loss))

  result = {"perplexity":perplexity}

  output_eval_file = os.path.join('./output-large',"eval_results.txt")

  with open(output_eval_file,"w") as writer:
    for key in sorted(result.keys()):
      writer.write("%s = %s\n" % (key,str(result[key])))

  return result


In [ ]:
def main(df_trn, df_val):
    cache_dir = 'cached'
    model_name = 't5-small'
    output_dir = './output-large'

    # Load the model and tokenizer
    config = AutoConfig.from_pretrained(model_name, cache_dir=cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        model_name,
        from_tf=False,
        config=config,
        cache_dir=cache_dir,
    )
    model.to(device)

    # Prepare the dataset
    train_dataset = load_and_cache_examples(tokenizer, df_trn, df_val, evaluate=False)

    # Train the model
    global_step, tr_loss = train(train_dataset, model, tokenizer)

    # Save the model and tokenizer
    os.makedirs(output_dir, exist_ok=True)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Reload the model and tokenizer
    model = AutoModelWithLMHead.from_pretrained(output_dir)
    tokenizer = AutoTokenizer.from_pretrained(output_dir)
    model.to(device)

    # Evaluate the model
    results = {}
    result = evaluate(model, tokenizer, df_trn, df_val)
    result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
    results.update(result)

    return results


In [ ]:
main(trn_df, val_df)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1731: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/189 [00:00<?, ?it/s]

Iteration:   0%|          | 0/189 [00:00<?, ?it/s]

Iteration:   0%|          | 0/189 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/189 [00:00<?, ?it/s]

Iteration:   0%|          | 0/189 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1731: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Evaluating:   0%|          | 0/189 [00:00<?, ?it/s]

{'perplexity_7560': tensor(76.1614)}

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Initialize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Initialize chat history
chat_history_ids = None

for step in range(5):
    # Encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input = input(">> User: ")
    new_user_input_ids = tokenizer.encode(new_user_input + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input tokens to the chat history
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
    else:
        bot_input_ids = new_user_input_ids

    # Generate a response while limiting the total chat history to 1000 tokens
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=150,
        pad_token_id=tokenizer.pad_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=50,
        top_p=0.85,
        temperature=0.8
    )

    # Pretty print last output tokens from bot
    bot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print(f"RickBot: {bot_response}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


>> User: hello rick
RickBot: 
>> User: hello
RickBot: 
>> User: can you answer me 
RickBot: 
>> User: fuck off
RickBot: 
>> User: which 
RickBot: 
